<a href="https://colab.research.google.com/github/thedarsideofit/finance_python_practices/blob/main/prophet_ticker_forecasting_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main Source

https://towardsdatascience.com/apple-stock-and-bitcoin-price-predictions-using-fbs-prophet-for-beginners-python-96d5ec404b77?gi=962fdc51375b

### Changepoint Detection
https://facebook.github.io/prophet/docs/trend_changepoints.html#automatic-changepoint-detection-in-prophet

### Plotly In Colab
https://colab.research.google.com/github/binnisb/blog/blob/master/_notebooks/2020-04-02-Plotly-in-lab.ipynb#scrollTo=fblKrSaCeCz3


In [ ]:
pip install alphavantage_api

In [ ]:
pip install alpha_vantage

In [ ]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import wordcloud
from alphaVantageAPI.alphavantage import AlphaVantage 

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
######### CHANGE THE TICKER HERE #########
ticker = 'GOOGL'
######### CHANGE THE TICKER HERE #########
ts = TimeSeries(key='Your Access Key',output_format='pandas')
stock, meta_data = ts.get_daily(symbol=ticker,outputsize='full')
stock.head()

In [ ]:
#Write/Read from a file.
stock.to_csv ('/content/drive/My Drive/Colab Notebooks/Finanzas/' + ticker + 'pricedata.csv', index = 'date', header=True)
stock = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Finanzas/' + ticker + 'pricedata.csv')
stock.head()

In [ ]:
from alpha_vantage.techindicators import TechIndicators
import matplotlib.pyplot as plt
ti = TechIndicators(key='NLRHFAQBGU5WGPQH',output_format='pandas')
data, meta_data = ti.get_sma(symbol=ticker,interval='daily', time_period=60,series_type = 'close')
data.plot()
plt.show()

In [ ]:
from alpha_vantage.techindicators import TechIndicators
import matplotlib.pyplot as plt
ti = TechIndicators(key='Youraccesskey',output_format='pandas')
data, meta_data = ti.get_rsi(symbol=ticker,interval='daily', time_period=60,series_type = 'close')
data.plot()
plt.show()

In [ ]:
#Convert the dataframe into the prophet expected format.
stock.rename(columns={'4. close': 'y', 'date': 'ds'},inplace=True)
stock['ds'] = pd.to_datetime(stock['ds'])
# To avoid the peaks we're gonna use the log
stock['y']= np.log(stock['y']).astype(float)
stock.head()

In [ ]:
#Fit the model 
df_prophet = Prophet(changepoint_prior_scale=0.0191, daily_seasonality=True)
df_prophet.fit(stock)

In [ ]:
fcast_time= 365   # 1 year
df_forecast = df_prophet.make_future_dataframe(periods= fcast_time, freq='D')
df_forecast.tail(10)

In [ ]:
# Forecasting - call the method predict
df_forecast = df_prophet.predict(df_forecast)
df_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper'] ]
df = df_forecast[df_forecast['ds']< '2020-11-30']
# Convert the value to see the price
df['yhat'] = np.exp(df['yhat'])
df['yhat_lower'] = np.exp(df['yhat_lower'])
df['yhat_upper'] = np.exp(df['yhat_upper'])
df[['ds','yhat', 'yhat_lower', 'yhat_upper'] ].to_csv ('/content/drive/My Drive/Colab Notebooks/Finanzas/' + ticker + 'pricedata-predicted.csv', index = 'ds', header=True)
df[['ds','yhat', 'yhat_lower', 'yhat_upper'] ].tail(30)

In [ ]:
fig = df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = ticker + ' Price')
fig.show()

In [ ]:
fig2 = df_prophet.plot_components(df_forecast)
plt.show()

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

fig = df_prophet.plot(df_forecast)
a = add_changepoints_to_plot(fig.gca(), df_prophet, df_forecast)


In [ ]:
from fbprophet.plot import plot_plotly
from IPython.display import HTML
fig = plot_plotly(df_prophet, df_forecast)  # This returns a plotly Figure
HTML(fig.to_html())

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(df_prophet, initial='730 days', period='180 days', horizon = '30 days')
df_cv.tail()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric = 'mse')